In [61]:
#Modules
import os
import librosa
import h5py
import sklearn
import numpy as np

## Training Data 

In [62]:
training_data_dir='/media/data_disk/datasets/TIMIT Dataset/timit/TIMIT/TRAIN/'

### Saving the directories for Training 

In [63]:
f1=open('train_words.txt','w')
f2=open('train_phn.txt','w')
f3=open('train_wav.txt','w')
for root,directory,files in os.walk(training_data_dir):
    if len(files)==0:
        continue
    else:
        for i in files:
            #print i.split('.')[1]
            if i.split('.')[1]=='WAV':
                f3.write(str(root)+'/'+str(i)+'\n')
            elif i.split('.')[1]=='PHN':
                f2.write(str(root)+'/'+str(i)+'\n')
            elif i.split('.')[1]=='TXT':
                f1.write(str(root)+'/'+str(i)+'\n')

f1.close()
f2.close()
f3.close()

### Reading and Processing Train Phonemes File

In [97]:
h5f=h5py.File('timit_files/train_phenome.h5','w')

phn_dict={}
with open('train_phn.txt','r') as f1:
    for path in f1:
        path=path.rstrip()
        word=path.split('/')[9]+'_'+path.split('/')[10].split('.')[0]
        phn_list=[]
        with open(path,'r') as f2:
            for phn in f2:
                phn_list.append(phn.split()[2])
            #phn_dict[word]=phn_list
            #print phn_list
            h5f.create_dataset(word,data=phn_list)
h5f.close()

In [7]:
#remove the h#
#turns out you need the h#.....ignore this function/cell
'''
h5f=h5py.File('timit_files/train_phenome_cleaned.h5','w')

with open('audio_key.txt','r') as f:
    with h5py.File('timit_files/train_phenome.h5', 'r') as h5:
        for line in f:
            line=line.rstrip()
            #print line
            phn=h5[line][:]
            phn=np.delete(phn,0)
            phn=np.delete(phn,-1)
            h5f.create_dataset(line,data=phn)
h5f.close()
'''

"\nh5f=h5py.File('timit_files/train_phenome_cleaned.h5','w')\n\nwith open('audio_key.txt','r') as f:\n    with h5py.File('timit_files/train_phenome.h5', 'r') as h5:\n        for line in f:\n            line=line.rstrip()\n            #print line\n            phn=h5[line][:]\n            phn=np.delete(phn,0)\n            phn=np.delete(phn,-1)\n            h5f.create_dataset(line,data=phn)\nh5f.close()\n"

In [98]:
#extract all the phonemes types and store them into a list

#array that will hold all the phonemes
master_phn_list=[]

with open('audio_key.txt','r') as f:
    with h5py.File('timit_files/train_phenome.h5', 'r') as h5:
        for line in f:
            line=line.rstrip()
            phn=h5[line][:]
            for i in phn:
                if i not in master_phn_list:
                    master_phn_list.append(i)

                    
#add blank for ctc 
master_phn_list.append('_')

print master_phn_list
len(master_phn_list)

['h#', 'jh', 'ao', 'r', 'dcl', 's', 'eh', 'l', 'd', 'ax', 'm', 'w', 'aa', 'tcl', 'ch', 'ix', 'z', 'ey', 't', 'ay', 'ux', 'v', 'f', 'ng', 'ae', 'bcl', 'b', 'kcl', 'k', 'n', 'iy', 'pcl', 'p', 'y', 'ah', 'nx', 'epi', 'ow', 'dx', 'ih', 'oy', 'gcl', 'g', 'dh', 'sh', 'hv', 'axr', 'q', 'zh', 'el', 'er', 'hh', 'ax-h', 'uh', 'th', 'en', 'aw', 'pau', 'em', 'uw', 'eng', '_']


62

In [100]:
#store the phenome master list array
h5f=h5py.File('timit_files/phoneme_list.h5','w')
h5f.create_dataset("list_phn",data=master_phn_list)
h5f.close()

In [101]:
#insert spaces and encode the phn with a integer key to an array list
h5f=h5py.File('timit_files/phoneme_list_encode.h5','w')

with open('audio_key.txt','r') as f:
    with h5py.File('timit_files/train_phenome.h5', 'r') as h5:
        for line in f:
            line=line.rstrip()
            phn_space=[]
            phn=h5[line][:]
            for i in phn:
                phn_space.append(master_phn_list.index('_'))
                phn_space.append(master_phn_list.index(i))
            phn_space.append(master_phn_list.index('_'))
            h5f.create_dataset(line,data=phn_space)
h5f.close()

In [102]:
#determine the largest phoneme 
max_length=0

with open('audio_key.txt','r') as f:
    with h5py.File('timit_files/phoneme_list_encode.h5', 'r') as h5:
        for line in f:
            line=line.rstrip()
            if len(h5[line][:])>max_length:
                max_length=len(h5[line][:])
print max_length

151


In [103]:
#all phonemes are padded with spaces and of the same length
h5f=h5py.File('timit_files/phoneme_list_encode_space_padded.h5','w')

with open('audio_key.txt','r') as f:
    with h5py.File('timit_files/phoneme_list_encode.h5', 'r') as h5:
        for line in f:
            line=line.rstrip()
            phn=list(h5[line][:])
            phn_len=len(phn)
            if phn_len<max_length:
                for i in range(max_length-phn_len):
                    phn.append(master_phn_list.index('_'))
            h5f.create_dataset(line,data=phn)
h5f.close()

### Reading and Processing Train Word File

In [13]:
h5f=h5py.File('timit_files/train_words.h5','w')
ctr=0

word_dict={}
with open('train_words.txt','r') as f1:
    for path in f1:
        path=path.rstrip()
        word=path.split('/')[9]+'_'+path.split('/')[10].split('.')[0]
        #print word
        word_list=[]
        with open(path,'r') as f2:
            for sent in f2:
                word_list.append(sent.split()[2:])
            #print word_list
            h5f.create_dataset(word,data=word_list)
h5f.close()

### Reading and Processing Train Wav File 

In [12]:
#reading audio files and converting them to numpy arrays
with h5py.File('timit_files/train_audio.h5', 'w') as h5:
    with open('train_wav.txt','r') as f1:
        for path in f1:
            path=path.rstrip()
            f_name=path.split('/')[9]+'_'+path.split('/')[10].split('.')[0]
            print f_name
            x,sr=librosa.load(path)
            mfcc_mat=librosa.feature.mfcc(x,sr,n_mfcc=55)
            scaled=sklearn.preprocessing.scale(mfcc_mat,axis=1)
            h5.create_dataset(f_name,data=scaled)

FCLT0_SX358
FCLT0_SX88
FCLT0_SA2
FCLT0_SA1
FCLT0_SI1438
FCLT0_SX268
FCLT0_SX178
FCLT0_SX448
FCLT0_SI808
FCLT0_SI2068
MMPM0_SI2321
MMPM0_SX341
MMPM0_SI1691
MMPM0_SX71
MMPM0_SA2
MMPM0_SI1061
MMPM0_SA1
MMPM0_SX251
MMPM0_SX431
MMPM0_SX161
FNKL0_SX262
FNKL0_SA2
FNKL0_SA1
FNKL0_SX442
FNKL0_SI892
FNKL0_SI1522
FNKL0_SX172
FNKL0_SX82
FNKL0_SX196
FNKL0_SI2152
MRLK0_SX303
MRLK0_SX213
MRLK0_SX393
MRLK0_SX33
MRLK0_SA2
MRLK0_SI843
MRLK0_SA1
MRLK0_SI2140
MRLK0_SI1468
MRLK0_SX123
FKLH0_SX87
FKLH0_SA2
FKLH0_SA1
FKLH0_SI1887
FKLH0_SX177
FKLH0_SI627
FKLH0_SX447
FKLH0_SI1257
FKLH0_SX267
FKLH0_SX357
MMEA0_SI2018
MMEA0_SX308
MMEA0_SX218
MMEA0_SA2
MMEA0_SI1388
MMEA0_SA1
MMEA0_SX128
MMEA0_SI758
MMEA0_SX38
MMEA0_SX398
MTCS0_SI1972
MTCS0_SX262
MTCS0_SI712
MTCS0_SA2
MTCS0_SA1
MTCS0_SX442
MTCS0_SI2265
MTCS0_SX172
MTCS0_SX82
MTCS0_SX352
FCEG0_SI1878
FCEG0_SX168
FCEG0_SA2
FCEG0_SA1
FCEG0_SX438
FCEG0_SX348
FCEG0_SI618
FCEG0_SX78
FCEG0_SX258
FCEG0_SI1248
MKDD0_SX397
MKDD0_SA2
MKDD0_SI2197
MKDD0_SA1
MKDD0_SI937
MKDD0_

In [14]:
#calculate the largest frame rate 
max_length=0
with open('audio_key.txt','r') as f:
    with h5py.File('timit_files/train_audio.h5', 'r') as h5:
        for name in f:
            name=name.rstrip()
            audio_arr=h5[name][:]
            if max_length<audio_arr.shape[1]:
                max_length=audio_arr.shape[1]
print max_length

336


In [15]:
h5f=h5py.File('timit_files/train_audio_zero_padded.h5','w')

zeros=np.zeros((55,1))
with open('audio_key.txt','r') as f:
    with h5py.File('timit_files/train_audio.h5', 'r') as h5:
        for name in f:
            name=name.rstrip()
            audio_arr=h5[name][:]
            if audio_arr.shape[1]<max_length:
                for i in range(max_length-audio_arr.shape[1]):
                    audio_arr=np.append(audio_arr,zeros,axis=1)
            h5f.create_dataset(name,data=audio_arr)
            
h5f.close()

## Testing Data

In [16]:
test_data_dir='/media/data_disk/datasets/TIMIT Dataset/timit/TIMIT/TEST'

### Saving the directories for Testing

In [72]:
f1=open('timit_files/test_words.txt','w')
f2=open('timit_files/test_phn.txt','w')
f3=open('timit_files/test_wav.txt','w')
for root,directory,files in os.walk(test_data_dir):
    if len(files)==0:
        continue
    else:
        for i in files:
            if i.split('.')[1]=='WAV':
                f3.write(str(root)+'/'+str(i)+'\n')
            elif i.split('.')[1]=='PHN':
                f2.write(str(root)+'/'+str(i)+'\n')
            elif i.split('.')[1]=='TXT':
                f1.write(str(root)+'/'+str(i)+'\n')

f1.close()
f2.close()
f3.close()



In [85]:
h5f.close()

## Reading and Processing Testing Data 

### Phonemes Extraction/ Processing

In [104]:
h5f=h5py.File('timit_files/test_phenome.h5','w')

with open('timit_files/test_phn.txt','r') as f1:
    for path in f1:
        path=path.rstrip()
        word=path.split('/')[9]+"_"+path.split('/')[10].split('.')[0]
        phn_list=[]
        with open(path,'r') as f2:
            for phn in f2:
                phn_list.append(phn.split()[2])
        h5f.create_dataset(word,data=phn_list)
h5f.close()

In [105]:
#insert spaces and encode the phn with a integer key to an array list
h5f=h5py.File('timit_files/test_phoneme_list_encode.h5','w')

with open('timit_files/test_audio_key.txt','r') as f:
    with h5py.File('timit_files/test_phenome.h5', 'r') as h5:
        for line in f:
            line=line.rstrip()
            phn_space=[]
            phn=h5[line][:]
            for i in phn:
                phn_space.append(master_phn_list.index('_'))
                phn_space.append(master_phn_list.index(i))
            phn_space.append(master_phn_list.index('_'))
            h5f.create_dataset(line,data=phn_space)
h5f.close()

In [106]:
#determine the largest phoneme 
max_length=0

with open('timit_files/test_audio_key.txt','r') as f:
    with h5py.File('timit_files/test_phoneme_list_encode.h5', 'r') as h5:
        for line in f:
            line=line.rstrip()
            if len(h5[line][:])>max_length:
                max_length=len(h5[line][:])
print max_length

147


In [113]:
#all phonemes are padded with spaces and of the same length
h5f=h5py.File('timit_files/test_phoneme_list_encode_space_padded.h5','w')

with open('timit_files/test_audio_key.txt','r') as f:
    with h5py.File('timit_files/test_phoneme_list_encode.h5', 'r') as h5:
        for line in f:
            line=line.rstrip()
            phn=list(h5[line][:])
            phn_len=len(phn)
            if phn_len<max_length:
                for i in range(max_length-phn_len):
                    phn.append(master_phn_list.index('_'))
            h5f.create_dataset(line,data=phn)
h5f.close()

### Word Extraction 

In [94]:
h5f=h5py.File('timit_files/test_words.h5','w')
ctr=0

word_dict={}
with open('train_words.txt','r') as f1:
    for path in f1:
        path=path.rstrip()
        word=path.split('/')[9]+'_'+path.split('/')[10].split('.')[0]
        #print word
        word_list=[]
        with open(path,'r') as f2:
            for sent in f2:
                word_list.append(sent.split()[2:])
            h5f.create_dataset(word,data=word_list)
h5f.close()

### Reading and Processing Testing Audio Files 

In [95]:
with h5py.File('timit_files/test_audio.h5', 'w') as h5:
    with open('timit_files/test_wav.txt','r') as f1:
        for path in f1:
            path=path.rstrip()
            f_name=path.split('/')[9]+'_'+path.split('/')[10].split('.')[0]
            print f_name
            x,sr=librosa.load(path)
            mfcc_mat=librosa.feature.mfcc(x,sr,n_mfcc=55)
            scaled=sklearn.preprocessing.scale(mfcc_mat,axis=1)
            h5.create_dataset(f_name,data=scaled)

MJTC0_SX290
MJTC0_SA2
MJTC0_SA1
MJTC0_SI1460
MJTC0_SX380
MJTC0_SX110
MJTC0_SI2090
MJTC0_SI830
MJTC0_SX20
MJTC0_SX200
MPAM0_SX109
MPAM0_SA2
MPAM0_SA1
MPAM0_SX19
MPAM0_SI1961
MPAM0_SI1819
MPAM0_SI1189
MPAM0_SX379
MPAM0_SX199
MPAM0_SX289
MSLB0_SX383
MSLB0_SI1193
MSLB0_SI563
MSLB0_SX293
MSLB0_SA2
MSLB0_SA1
MSLB0_SX113
MSLB0_SX23
MSLB0_SX203
MSLB0_SI1823
FCMH1_SX413
FCMH1_SA2
FCMH1_SA1
FCMH1_SI863
FCMH1_SI1493
FCMH1_SX323
FCMH1_SX143
FCMH1_SI2123
FCMH1_SX233
FCMH1_SX53
FMLD0_SI2185
FMLD0_SX205
FMLD0_SX115
FMLD0_SA2
FMLD0_SA1
FMLD0_SX25
FMLD0_SI822
FMLD0_SX385
FMLD0_SI925
FMLD0_SX295
MRES0_SX137
MRES0_SA2
MRES0_SA1
MRES0_SI587
MRES0_SX227
MRES0_SX317
MRES0_SI1847
MRES0_SX407
MRES0_SI1217
MRES0_SX47
MJLN0_SI1449
MJLN0_SX99
MJLN0_SA2
MJLN0_SA1
MJLN0_SX279
MJLN0_SX189
MJLN0_SI2079
MJLN0_SI819
MJLN0_SX369
MJLN0_SX9
MDAW1_SI2083
MDAW1_SA2
MDAW1_SI1453
MDAW1_SA1
MDAW1_SX373
MDAW1_SX283
MDAW1_SI823
MDAW1_SX13
MDAW1_SX193
MDAW1_SX103
FJSJ0_SX224
FJSJ0_SX404
FJSJ0_SA2
FJSJ0_SA1
FJSJ0_SX44
FJSJ0_SI854

In [114]:
#calculate the largest frame rate 
max_length=0
with open('timit_files/test_audio_key.txt','r') as f:
    with h5py.File('timit_files/test_audio.h5', 'r') as h5:
        for name in f:
            name=name.rstrip()
            audio_arr=h5[name][:]
            if max_length<audio_arr.shape[1]:
                max_length=audio_arr.shape[1]
print max_length

327


In [118]:
h5f=h5py.File('timit_files/test_audio_zero_padded.h5','w')

zeros=np.zeros((55,1))
with open('timit_files/test_audio_key.txt','r') as f:
    with h5py.File('timit_files/test_audio.h5', 'r') as h5:
        for name in f:
            name=name.rstrip()
            audio_arr=h5[name][:]
            if audio_arr.shape[1]<max_length:
                for i in range(max_length-audio_arr.shape[1]):
                    audio_arr=np.append(audio_arr,zeros,axis=1)
            h5f.create_dataset(name,data=audio_arr)
            
h5f.close()